In [1]:
import tensorflow as tf
import numpy as np
import scipy.stats
import scipy.io
import scipy.sparse
from scipy.io import loadmat
import pandas as pd
import tensorflow_probability as tfp
tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_iris
data = load_iris(True)[0]

We now standardise the data:

In [3]:
xfull = ((data - np.mean(data,0))/np.std(data,0)).astype(np.float32)
n = xfull.shape[0] # number of observations
p = xfull.shape[1] # number of feat*ures

We will remove uniformy at random 10% of the data. This corresponds to a *missing completely at random (MCAR)* scenario.

In [4]:
perc_miss = 0.1 # 50% of missing data
xmiss = np.copy(xfull)
xmiss_flat = xmiss.flatten()
miss_pattern = np.random.choice(n*p, np.floor(n*p*perc_miss).astype(np.int), replace=False)
xmiss_flat[miss_pattern] = np.nan 
xmiss = xmiss_flat.reshape([n,p]) # in xmiss, the missing values are represented by nans
mask = np.isfinite(xmiss) # binary mask that indicates which values are missing

In [5]:
xmiss

array([[-9.0068120e-01,  1.0190043e+00, -1.3402265e+00, -1.3154444e+00],
       [-1.1430169e+00,            nan, -1.3402265e+00, -1.3154444e+00],
       [-1.3853526e+00,  3.2841405e-01, -1.3970640e+00, -1.3154444e+00],
       [-1.5065205e+00,  9.8217286e-02, -1.2833891e+00, -1.3154444e+00],
       [-1.0218490e+00,  1.2492011e+00, -1.3402265e+00, -1.3154444e+00],
       [-5.3717756e-01,  1.9397914e+00, -1.1697142e+00, -1.0521799e+00],
       [-1.5065205e+00,  7.8880757e-01, -1.3402265e+00, -1.1838121e+00],
       [-1.0218490e+00,            nan, -1.2833891e+00, -1.3154444e+00],
       [-1.7488563e+00, -3.6217624e-01,            nan, -1.3154444e+00],
       [-1.1430169e+00,  9.8217286e-02, -1.2833891e+00, -1.4470764e+00],
       [-5.3717756e-01,  1.4793979e+00, -1.2833891e+00, -1.3154444e+00],
       [-1.2641848e+00,  7.8880757e-01, -1.2265517e+00, -1.3154444e+00],
       [-1.2641848e+00, -1.3197948e-01, -1.3402265e+00, -1.4470764e+00],
       [-1.8700241e+00, -1.3197948e-01, -1.5107388e

We want to learn a Gaussian distribution.

In [6]:
mu = tf.Variable(tf.ones([p]), dtype=tf.float32)
log_sigma_diag = tf.Variable(tf.zeros([p]), dtype=tf.float32) #log sigma of the gaussian

In [7]:
@tf.function
def log_likelihood(x):
  sigma_diag = tf.exp(log_sigma_diag)
  p_x = tfd.MultivariateNormalDiag(loc = mu, scale_diag = sigma_diag)
  return(p_x.log_prob(x))

In [8]:
params = [mu] + [log_sigma_diag]

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [9]:
def train_step(data):
  with tf.GradientTape() as tape: # the gradient tape saves all the step that needs to be saved fopr automatic differentiation
    loss = -log_likelihood(data)  # the loss is the average negative log likelihood
  gradients = tape.gradient(loss, params)  # here, the gradient is automatically computed
  optimizer.apply_gradients(zip(gradients, params))  # Adam iteration

In [10]:
train_data_complete = tf.data.Dataset.from_tensor_slices(xfull).shuffle(p).batch(1) 

In [11]:
EPOCHS = 1000

for epoch in range(1,EPOCHS+1):
  for data in train_data_complete:
    train_step(data) # Adam iteration
  if (epoch % 100) == 1:
    ll_train = tf.reduce_mean(log_likelihood(xfull))
    print('Epoch  %g' %epoch)
    print('Training log-likelihood %g' %ll_train.numpy())
    print('Mean %g')
    tf.print(mu)
    print('-----------')

Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Epoch  1
Training log-likelihood -7.62363
Mean %g
[0.991500497 0.988992572 0.992242455 0.992015123]
-----------
Epoch  101
Training log-likelihood -5.84347
Mean %g
[0.287656546 0.282061607 0.291251 0.29137516]
-----------
Epoch  201
Training log-likelihood -5.68566
Mean %g
[0.068391487 0.0631356165 0.0721252114 0.0727692619]
-----------
Epoch  301
Training log-likelihood -5.67655
Mean %g
[0.0187689811 0.0136054559 0.0224475972 0.0232414473]
-----------
Epoch  401
Training log-likelihood -5.67593
Mean %g
[0.00767116714 0.00253115944 0.0113104694 0.0121313715]
-----------
Epoch  501
Training log-likelihood -5.67586
Mean %g
[0.0051474059 -2.12646901e-05 0.00879643112 0.00964250136]
-----------
Epoch  601
Training log-likelihood -5.67584
Mean %g
[0.00462956168 -0.000452933687 0.00828512944 0.00909783598]
-----------
Epoch  701
Training log-likelihood -5.67584
Mean %g
[0.00457425741 -0.000640362 0.0081681

In [ ]:
@tf.function
def log_likelihood_incomplete(x):
  sigma_diag = tf.exp(log_sigma_diag)
  p_x = tfd.MultivariateNormalDiag(loc = mu, scale_diag = sigma_diag)
  return(p_x.log_prob(x))

In [ ]:
def train_step(data):
  with tf.GradientTape() as tape: # the gradient tape saves all the step that needs to be saved fopr automatic differentiation
    loss = -log_likelihood(data)  # the loss is the average negative log likelihood
  gradients = tape.gradient(loss, params)  # here, the gradient is automatically computed
  optimizer.apply_gradients(zip(gradients, params))  # Adam iteration

In [ ]:
train_data_complete = tf.data.Dataset.from_tensor_slices(xfull).shuffle(p).batch(1) 